# Stream Cipher

In [43]:
from streamcipher import LFSR
from streamcipher import berlekamp_massey
from streamcipher import ShrinkingGenerator, SelfShrinkingGenerator

from itertools import islice

## LFSR

### testing

Test LFSR implementation (all attributes and mehtods) with the polynomial `poly = [4, 1, 0]`.

In [44]:
def lfsr_generator(poly_list, state_list=None, berk=False):
    lfsr = LFSR(poly_list, state_list)

    if not berk :
        print('\nstate     b fb')
        for _ in islice(lfsr, 10):
            print(lfsr)

    if berk :
        full_cycle = lfsr.cycle()
        print(f'\nfull LFSR cycle -> {full_cycle}')

        P = berlekamp_massey(full_cycle)

        print(f'\nBerlekamp-Massey algorithm -> {P}')
        P.reverse()
        nonzero = [i for i, e in enumerate(P) if e != 0]
        poly_list = [f'x^{d}' for d in nonzero if d!= 0]
        poly_list.reverse()
        poly = "+".join(poly_list) + "+1"
        print(poly)

In [45]:
# We could make list comprehension but it's not very readable
def poly_to_list(degrees):
    """ Convert a polynomial to a list of taps for an LFSR """
    length = max(degrees) + 1
    taps = [1 if i in degrees else 0 for i in range(length)]
    return taps[::-1]

In [46]:
poly_list = poly_to_list([4, 1, 0])
print(f"Poly list: {poly_list}")
lfsr_generator(poly_list)

Poly list: [1, 0, 0, 1, 1]
Initial state: [1, 1, 1, 1, 1]

state     b fb
[0, 1, 1, 1, 1] 1 0
[0, 0, 1, 1, 1] 1 0
[0, 0, 0, 1, 1] 1 1
[1, 0, 0, 0, 1] 1 0
[0, 1, 0, 0, 0] 0 0
[0, 0, 1, 0, 0] 0 1
[1, 0, 0, 1, 0] 0 1
[1, 1, 0, 0, 1] 1 0
[0, 1, 1, 0, 0] 0 1
[1, 0, 1, 1, 0] 0 0


## Berlekamp Massey

### testing

Test the implementation Berlekamp-Massy Algorithm with the sequence produced by the following LFSR:
- `poly = [3, 1, 0]` and `state = '\x06'`
- `poly = [5, 2, 1, 0]` and `state = '\x05'`
- `poly = [96, 81, 17, 15, 0]` and `state = b'streamcipher'`

In [47]:
poly_list = poly_to_list([3, 1, 0])
state = int.from_bytes(b'\x06', byteorder='big')
lfsr_generator(poly_list, state, True)


Initial state: [0, 1, 1, 0]

full LFSR cycle -> [1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1]

Berlekamp-Massey algorithm -> [0, 1, 3, 4, 5, 7, 8, 9, 10, 13, 16, 17, 18]
x^11+x^10+x^9+x^8+x^7+x^6+x^5+x^4+x^3+x^2+x^1+1


In [48]:
poly_list = poly_to_list([5, 2, 0])
state = int.from_bytes(b'\x05', byteorder='big')
lfsr_generator(poly_list, state, True)

Initial state: [0, 0, 0, 1, 0, 1]

full LFSR cycle -> [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0]

Berlekamp-Massey algorithm -> [0, 2, 4, 6, 8, 9, 17, 18, 19, 22]
x^8+x^7+x^6+x^5+x^4+x^3+x^2+x^1+1


In [49]:
poly_list = poly_to_list([96, 81, 17, 15, 0])
state = int.from_bytes(b'streamcipher', byteorder='big')
#lfsr_generator(poly_list, state, True)

Load `'binary_sequence'` and find the polynomial corresponding to the shortest LFSR that can produce that sequence.

In [50]:
# Loading the binary_sequence.bin file
with open('binary_sequence.bin', 'rb') as f:
    data = f.read()

# Finding the polynomial corresponding to the shortest LFSR that generates the sequence
#lfsr_generator(data, berk=True)

## LFSR-based Generator (Shrinking or Self-Shrinking Generator)

### testing

Test Generator implementation (all attributes and methods).

In [51]:
# Testing the Shrinking Generator
print("\nShrinking Generator")
poly_list1 = poly_to_list([3, 2, 0])
poly_list2 = poly_to_list([3, 1, 0])
state1 = int.from_bytes(b'\x06', byteorder='big')
state2 = int.from_bytes(b'\x05', byteorder='big')

sg = ShrinkingGenerator(poly_list1, poly_list2, state1, state2)

# Test all attributes
print(f'LFSRA: {sg.lfsrA}')
print(f'LFSRS: {sg.lfsrS}')
print(f'output: {sg.output}')

# Testing the next method
print(next(sg))
print(next(sg))
print(next(sg))

# Testing the iter method
iterator = iter(sg)
print(next(iterator))
print(next(iterator))
print(next(iterator))





Shrinking Generator
Initial state: [0, 1, 1, 0]
Initial state: [0, 1, 0, 1]
LFSRA: [0, 1, 1, 0] [0] 1
LFSRS: [0, 1, 0, 1] [0] 1
output: None
0
1
0
1
0
0


In [52]:
# Testing the Self Shrinking Generator
print("\nSelf Shrinking Generator")
poly_list = poly_to_list([3, 2, 0])
sbit = 4
state = int.from_bytes(b'\x06', byteorder='big')

ssg = SelfShrinkingGenerator(poly_list, sbit, state)

print(f'\nstate     b fb')
for _ in islice(ssg, 10):
    print(ssg)


Self Shrinking Generator
Initial state: [0, 1, 1, 0]

state     b fb


Decrypt the ciphertext in `ciphertext_shrinking.bin` or `ciphertext_selfshrinking.bin`.

In [53]:
# Transform the name group into a bytes object (big endian)
group_name = b'EncryptionEagles'

# Iterate over the 8 bits of the byte and shift the byte i times to the right and check if the least significant bit is 1.
group_bits = [bool(byte & (1 << i)) for byte in group_name for i in range(8)]

# Comptute the parity bit
parity = sum(group_bits) % 2
print(f'Parity bit: {parity}')


Parity bit: 1


In [54]:
polyA = [16, 15, 12, 10, 0]
polyS = [24, 11, 5, 2, 0]
stateA = b'\xc5\xd7'
stateS = b'\x14\x84\xf8'

In [55]:
poly = [32, 16, 7, 2, 0]
selection_bit = 4
state = b'mJ\x9by'

In [56]:
# Choosing the assigned generator depending on the parity bit
if parity == 0:
    sg = ShrinkingGenerator(polyA, polyS, int.from_bytes(stateA, byteorder='big'), int.from_bytes(stateS, byteorder='big'))
else:
    ssg = SelfShrinkingGenerator(poly, selection_bit, int.from_bytes(state, byteorder='big'))

Initial state: [1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1]


## Conclusion